In [1]:
!pip install hfst_dev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 13.6 MB/s eta 0:00:00


In [2]:
import hfst_dev

In [15]:
!wget https://raw.githubusercontent.com/kikis202/NLP_MD1/main/lexicon.lexc

--2024-03-28 14:46:03--  https://raw.githubusercontent.com/kikis202/NLP_MD1/main/lexicon.lexc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 973 [text/plain]
Saving to: ‘lexicon.lexc’

lexicon.lexc        100%[===================>]     973  --.-KB/s    in 0s      

2024-03-28 14:46:03 (85.7 MB/s) - ‘lexicon.lexc’ saved [973/973]



# FST

## FST izveide

### Leksikons no faila


In [16]:
generator = hfst_dev.compile_lexc_file('lexicon.lexc')

### Līdzskaņu mija.
Ievieto `j` pirms `^M`

In [18]:
InsertJ = hfst_dev.regex('[..] -> j || [b | m | p | v | f] _ "^" M')

### Izņēmumi
Vārdam `žirafe` nenotiek līdzskaņu mija daudzskaitļa ģenitīvā

In [19]:
Exceptions = hfst_dev.regex('[[{žiraf} "+N" "+Pl" "+Gen" .x. {žirafu}] ]')

### Aizstāšana
Pie mijas `l` tiek aizvietots ar `ļ`

In [13]:
LReplacement = hfst_dev.regex('l -> ļ ||  _ "^" M')

### Dzēšana
Tiek dzēsti palīg-simboli

In [24]:
MCleanup = hfst_dev.regex('"^" M -> 0')
Cleanup = hfst_dev.regex('"^" -> 0')

### FST apvienošana
Pēc kārtas tiek pielietotas iepriekš definētās funkcijas

In [28]:
from hfst_dev import compose
Exceptions = hfst_dev.regex('[[{žiraf} "+N" "+Pl" "+Gen" .x. {žirafu}] ]')
cascade = compose((generator, InsertJ, LReplacement, MCleanup,Cleanup))

### Vārda ģenerēšana
Funkcija pārbauda vai vārdam ir izņēmumi, ja ir atgriež to, ja nav izpilda FST apvienojumu

In [36]:
def generate_word(input):
  exception = Exceptions.lookup(input)

  if len(exception) > 0:
    print(exception)
    return
    # return exception

  output = cascade.lookup(input)
  print(output)
  # return output

### Visu locījumu ģenerēšana

In [43]:
def generate_all(input):
  print("Vienskaitlis")
  generate_word(f'{input}+N+Sg+Nom')
  generate_word(f'{input}+N+Sg+Gen')
  generate_word(f'{input}+N+Sg+Dat')
  generate_word(f'{input}+N+Sg+Acc')
  generate_word(f'{input}+N+Sg+Loc')

  print()
  print("Daudzskaitlis")

  generate_word(f'{input}+N+Pl+Nom')
  generate_word(f'{input}+N+Pl+Gen')
  generate_word(f'{input}+N+Pl+Dat')
  generate_word(f'{input}+N+Pl+Acc')
  generate_word(f'{input}+N+Pl+Loc')

  print()


## Rezultāti

### Īpašie gadījumi

In [48]:
# izņēmums
generate_word("žiraf+N+Pl+Gen")

# aizvietošana
generate_word("pil+N+Pl+Gen")

(('žirafu', 0.0),)
(('piļu', 0.0),)


### 4\. deklanācijas vārdi

In [47]:
generate_all("fīč")
generate_all("galv")
generate_all("kafij")

Vienskaitlis
(('fīča', 0.0),)
(('fīčas', 0.0),)
(('fīčai', 0.0),)
(('fīču', 0.0),)
(('fīčā', 0.0),)

Daudzskaitlis
(('fīčas', 0.0),)
(('fīču', 0.0),)
(('fīčām', 0.0),)
(('fīčas', 0.0),)
(('fīčās', 0.0),)

Vienskaitlis
(('galva', 0.0),)
(('galvas', 0.0),)
(('galvai', 0.0),)
(('galvu', 0.0),)
(('galvā', 0.0),)

Daudzskaitlis
(('galvas', 0.0),)
(('galvu', 0.0),)
(('galvām', 0.0),)
(('galvas', 0.0),)
(('galvās', 0.0),)

Vienskaitlis
(('kafija', 0.0),)
(('kafijas', 0.0),)
(('kafijai', 0.0),)
(('kafiju', 0.0),)
(('kafijā', 0.0),)

Daudzskaitlis
(('kafijas', 0.0),)
(('kafiju', 0.0),)
(('kafijām', 0.0),)
(('kafijas', 0.0),)
(('kafijās', 0.0),)



### 5\. deklanācijas vārdi

In [49]:
generate_all("caurul")
generate_all("diagonāl")
generate_all("pel")
generate_all("žiraf")

Vienskaitlis
(('caurule', 0.0),)
(('caurules', 0.0),)
(('caurulei', 0.0),)
(('cauruli', 0.0),)
(('caurulē', 0.0),)

Daudzskaitlis
(('caurules', 0.0),)
(('cauruļu', 0.0),)
(('caurulēm', 0.0),)
(('caurules', 0.0),)
(('caurulēs', 0.0),)

Vienskaitlis
(('diagonāle', 0.0),)
(('diagonāles', 0.0),)
(('diagonālei', 0.0),)
(('diagonāli', 0.0),)
(('diagonālē', 0.0),)

Daudzskaitlis
(('diagonāles', 0.0),)
(('diagonāļu', 0.0),)
(('diagonālēm', 0.0),)
(('diagonāles', 0.0),)
(('diagonālēs', 0.0),)

Vienskaitlis
(('pele', 0.0),)
(('peles', 0.0),)
(('pelei', 0.0),)
(('peli', 0.0),)
(('pelē', 0.0),)

Daudzskaitlis
(('peles', 0.0),)
(('peļu', 0.0),)
(('pelēm', 0.0),)
(('peles', 0.0),)
(('pelēs', 0.0),)

Vienskaitlis
(('žirafe', 0.0),)
(('žirafes', 0.0),)
(('žirafei', 0.0),)
(('žirafi', 0.0),)
(('žirafē', 0.0),)

Daudzskaitlis
(('žirafes', 0.0),)
(('žirafu', 0.0),)
(('žirafēm', 0.0),)
(('žirafes', 0.0),)
(('žirafēs', 0.0),)



### 6\. deklanācijas vārdi

In [50]:
generate_all("ac")
generate_all("sird")
generate_all("ugun")
generate_all("pil")

Vienskaitlis
(('acs', 0.0),)
(('acs', 0.0),)
(('acij', 0.0),)
(('aci', 0.0),)
(('acī', 0.0),)

Daudzskaitlis
(('acis', 0.0),)
(('acu', 0.0),)
(('acīm', 0.0),)
(('acis', 0.0),)
(('acīs', 0.0),)

Vienskaitlis
(('sirds', 0.0),)
(('sirds', 0.0),)
(('sirdij', 0.0),)
(('sirdi', 0.0),)
(('sirdī', 0.0),)

Daudzskaitlis
(('sirdis', 0.0),)
(('sirdu', 0.0),)
(('sirdīm', 0.0),)
(('sirdis', 0.0),)
(('sirdīs', 0.0),)

Vienskaitlis
(('uguns', 0.0),)
(('uguns', 0.0),)
(('ugunij', 0.0),)
(('uguni', 0.0),)
(('ugunī', 0.0),)

Daudzskaitlis
(('ugunis', 0.0),)
(('ugunu', 0.0),)
(('ugunīm', 0.0),)
(('ugunis', 0.0),)
(('ugunīs', 0.0),)

Vienskaitlis
(('pils', 0.0),)
(('pils', 0.0),)
(('pilij', 0.0),)
(('pili', 0.0),)
(('pilī', 0.0),)

Daudzskaitlis
(('pilis', 0.0),)
(('piļu', 0.0),)
(('pilīm', 0.0),)
(('pilis', 0.0),)
(('pilīs', 0.0),)

